In [ ]:
from math import pi
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator
import matplotlib.pyplot as plt

# TASK 3a

def quantum_galton_board(n_layers):

  n_path = 2*(n_layers) - 1
  q = QuantumRegister(2*n_layers, 'q') # determined through the paper
  c = ClassicalRegister(n_path, 'c') # determined through the paper
  qc = QuantumCircuit(q, c)

  ctrl_qubit = 0 # this will be 0 as mentioned in the code provided in the paper

  p_qubits = range(1, 2*n_layers) # same reasoning here as the above
  midpoint = n_layers
  qc.x(midpoint)

  for layer in range(n_layers):
    qc.reset(ctrl_qubit) # reset q[0]
    qc.h(ctrl_qubit) # h q[0]

    for dest in range(midpoint - layer, midpoint + layer): # this is the range as each rounds' placements depends on the layer
      qc.cswap(ctrl_qubit, dest, dest + 1) # these are the processes as mentioned in sample code
      qc.cx(dest + 1, ctrl_qubit)

  for pos in p_qubits:
    qc.measure(pos, pos - 1)

  return qc

qc = quantum_galton_board(n_layers = 6)
simulator = AerSimulator()
result = simulator.run(qc, shots = 5000).result()
counts = result.get_counts()

plot_histogram(counts)
plt.show()

# TASK 3b
# the only thing that will be changing here is the gate as that determines the probability of the split
# since a hadamard was used for a 50/50 split, we will be using a rotation gate here instead to gear the split towards one side which will produce an exponential

def exponential_distribution(n_layers):

  n_path = 2*(n_layers) - 1
  q = QuantumRegister(2*n_layers, 'q')
  c = ClassicalRegister(n_path, 'c')
  qc = QuantumCircuit(q, c)

  ctrl_qubit = 0

  p_qubits = range(1, 2*n_layers)
  midpoint = n_layers
  qc.x(midpoint)

  for layer in range(n_layers):
    qc.reset(ctrl_qubit)
    qc.rx(pi/3.5, ctrl_qubit) # pi/3.5 gave the best exponential distribution - gate changed

    # for an exponential, all of this is kept the same
    for dest in range(midpoint - layer, midpoint + layer):
      qc.cswap(ctrl_qubit, dest, dest + 1)
      qc.cx(dest + 1, ctrl_qubit)

  for pos in p_qubits:
    qc.measure(pos, pos - 1)

  return qc

qc_exp = exponential_distribution(n_layers = 6)
result_exp = simulator.run(qc_exp, shots = 5000).result()
counts_exp = result_exp.get_counts()
plot_histogram(counts_exp)
plt.show()

# TASK 3c

def hadamard_quantum_walk(n_layers):

  n_path = 2*(n_layers) - 1
  q = QuantumRegister(2*n_layers, 'q')
  c = ClassicalRegister(n_path, 'c')
  qc = QuantumCircuit(q, c)

  ctrl_qubit = 0

  p_qubits = range(1, 2*n_layers)
  midpoint = n_layers
  qc.x(midpoint)

  for layer in range(n_layers):
    qc.reset(ctrl_qubit)
    qc.h(ctrl_qubit)

    for dest in range(1, n_path - 1):
      qc.cswap(ctrl_qubit, dest, dest + 1)
      qc.x(ctrl_qubit)

    for dest in range(1, n_path - 1):
      qc.cswap(ctrl_qubit, dest + 1, dest)
      qc.x(ctrl_qubit)

  for pos in p_qubits:
    qc.measure(pos, pos - 1)

  return qc

qc_hadamard = hadamard_quantum_walk(n_layers = 6)
result_hadamard = simulator.run(qc_hadamard, shots = 5000).result()
counts_hadamard = result_hadamard.get_counts()
plot_histogram(counts_hadamard)
plt.show()